In [25]:
import pandas as pd
import numpy as np 
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from catboost import CatBoostRegressor,CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import math
from rdkit.Chem import Descriptors
from rdkit import Chem
from mordred import Calculator,descriptors
import mordred
import rdkit
from rdkit import DataStructs

In [18]:
data = pd.read_csv("data/train_raw_data.csv",index_col=0)
data

,Inchi,Toxicity,Mol
0,InChI=1S/C10H19N5S/c1-6(2)11-8-13-9(12-7(3)4)1...,0,<rdkit.Chem.rdchem.Mol object at 0x131179e00>
1,InChI=1S/C12H10O4/c13-11(14)7-5-9-1-2-10(4-3-9...,0,<rdkit.Chem.rdchem.Mol object at 0x131179af0>
2,InChI=1S/C12H16O/c13-12-9-5-4-8-11(12)10-6-2-1...,0,<rdkit.Chem.rdchem.Mol object at 0x131179e70>
3,InChI=1S/C10H20O/c1-2-3-4-5-6-7-8-10-9-11-10/h...,0,<rdkit.Chem.rdchem.Mol object at 0x131179ee0>
4,InChI=1S/C16H13ClN2O/c1-19-14-8-7-12(17)9-13(1...,0,<rdkit.Chem.rdchem.Mol object at 0x131179f50>
...,...,...,...
7508,InChI=1S/C16H15F3N2O4/c1-8-12(15(22)25-3)13(14...,1,<rdkit.Chem.rdchem.Mol object at 0x130a90eb0>
7509,InChI=1S/C26H25NO4/c28-18-8-7-15-11-20-26(29)1...,1,<rdkit.Chem.rdchem.Mol object at 0x130a90f20>
7510,InChI=1S/C15H14N2O/c1-9-7-10(2)16-14(9)8-12-11...,1,<rdkit.Chem.rdchem.Mol object at 0x130a90f90>
7511,InChI=1S/C24H34N4O5S/c1-4-21-17(3)15-28(22(21)...,1,<rdkit.Chem.rdchem.Mol object at 0x130a91000>


In [19]:
test = pd.read_csv("data/external_test_raw_data.csv",index_col=0)
test["Toxicity"].value_counts()

Toxicity
0    543
1     71
Name: count, dtype: int64

In [20]:
def All_Mordred_descriptors(data):
    calc = Calculator(descriptors, ignore_3D=False)
    df = calc.pandas(data)
    return df

In [21]:

for i in range(len(data)):
    inchi = Chem.MolFromInchi(data["Inchi"][i])
    if inchi == None:
        print(i,":",data["Mol"][i],data["Toxicity"][i] )
        data.drop(index = i,inplace=True,axis=0)
#     else:
#         mol_list.append(All_Mordred_descriptors(inchi))
data.reset_index(inplace=True)
data.drop("index",axis = 1,inplace=True)

[19:07:08] WARNING: not removing hydrogen atom without neighbors
[19:07:08] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[19:07:08] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted



1322 : nan 0


[19:07:09] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match; Proton balance: Does not match)

[19:07:09] Explicit valence for atom # 0 Cl, 3, is greater than permitted
[19:07:09] ERROR: Explicit valence for atom # 0 Cl, 3, is greater than permitted



5624 : nan 0


[19:07:10] WARNING: not removing hydrogen atom without neighbors
[19:07:10] WARNING: not removing hydrogen atom without neighbors
[19:07:10] WARNING: not removing hydrogen atom without neighbors
[19:07:10] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[19:07:10] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[19:07:10] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Missing, Attachment points)



6454 : nan 0


[19:07:10] WARNING: not removing hydrogen atom without neighbors
[19:07:10] WARNING: not removing hydrogen atom without neighbors


In [22]:
data

,Inchi,Toxicity,Mol
0,InChI=1S/C10H19N5S/c1-6(2)11-8-13-9(12-7(3)4)1...,0,<rdkit.Chem.rdchem.Mol object at 0x131179e00>
1,InChI=1S/C12H10O4/c13-11(14)7-5-9-1-2-10(4-3-9...,0,<rdkit.Chem.rdchem.Mol object at 0x131179af0>
2,InChI=1S/C12H16O/c13-12-9-5-4-8-11(12)10-6-2-1...,0,<rdkit.Chem.rdchem.Mol object at 0x131179e70>
3,InChI=1S/C10H20O/c1-2-3-4-5-6-7-8-10-9-11-10/h...,0,<rdkit.Chem.rdchem.Mol object at 0x131179ee0>
4,InChI=1S/C16H13ClN2O/c1-19-14-8-7-12(17)9-13(1...,0,<rdkit.Chem.rdchem.Mol object at 0x131179f50>
...,...,...,...
7505,InChI=1S/C16H15F3N2O4/c1-8-12(15(22)25-3)13(14...,1,<rdkit.Chem.rdchem.Mol object at 0x130a90eb0>
7506,InChI=1S/C26H25NO4/c28-18-8-7-15-11-20-26(29)1...,1,<rdkit.Chem.rdchem.Mol object at 0x130a90f20>
7507,InChI=1S/C15H14N2O/c1-9-7-10(2)16-14(9)8-12-11...,1,<rdkit.Chem.rdchem.Mol object at 0x130a90f90>
7508,InChI=1S/C24H34N4O5S/c1-4-21-17(3)15-28(22(21)...,1,<rdkit.Chem.rdchem.Mol object at 0x130a91000>


In [23]:
for i in range(len(test)):
    inchi = Chem.MolFromInchi(test["Inchi"][i])
    if inchi == None:
        print(i,":",test["Mol"][i],test["Toxicity"][i] )
        data.drop(index = i,inplace=True,axis=0)
#     else:
#         mol_list.append(All_Mordred_descriptors(inchi))
test.reset_index(inplace=True)
test.drop("index",axis = 1,inplace=True)

In [24]:
mordred_data = All_Mordred_descriptors([Chem.MolFromInchi(i) for i in data["Inchi"]])
mordred_test = All_Mordred_descriptors([Chem.MolFromInchi(i) for i in test["Inchi"]])
mordred_data["Toxicity"] = data["Toxicity"]
mordred_test["Toxicity"] = test["Toxicity"]

[19:08:45] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match; Proton balance: Does not match)

[19:08:46] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Missing, Attachment points)

[19:08:46] WARNING: not removing hydrogen atom without neighbors
[19:08:46] WARNING: not removing hydrogen atom without neighbors
  2%|▊                                         | 153/7510 [00:13<07:19, 16.74it/s]

/Users/linyuheng/miniforge3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  3%|█▏                                        | 211/7510 [00:19<12:26,  9.77it/s]

/Users/linyuheng/miniforge3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  4%|█▌                                        | 277/7510 [00:23<07:23, 16.31it/s]

...

 97%|███████████████████████████████████████▊ | 7303/7510 [14:04<00:16, 12.44it/s][19:22:52] WARNING: not removing hydrogen atom without neighbors
[19:22:52] WARNING: not removing hydrogen atom without neighbors
[19:22:52] WARNING: not removing hydrogen atom without neighbors
[19:22:52] WARNING: not removing hydrogen atom without neighbors
[19:22:52] WARNING: not removing hydrogen atom without neighbors
[19:22:52] WARNING: not removing hydrogen atom without neighbors
 29%|████████████▋                              | 181/614 [00:14<00:26, 16.08it/s]

/Users/linyuheng/miniforge3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 53%|██████████████████████▌                    | 323/614 [00:30<00:18, 15.71it/s]

/Users/linyuheng/miniforge3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|███████████████████████████████████████████| 614/614 [00:53<00:00, 11.49it/s]


In [26]:
#morgan
fingerprints = []
for x in data["Inchi"]:
    mol = Chem.MolFromInchi(x)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    fp_arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, fp_arr)
    fp_arr = np.array([int(x) for x in fp_arr], dtype=np.float32)
    fingerprints.append(fp_arr)
morgan_data = pd.DataFrame(fingerprints)
morgan_data["Toxicity"] = data["Toxicity"]

[19:24:13] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match; Proton balance: Does not match)

[19:24:15] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Missing, Attachment points)

[19:24:15] WARNING: not removing hydrogen atom without neighbors
[19:24:15] WARNING: not removing hydrogen atom without neighbors


In [27]:
fingerprints = []
for x in test["Inchi"]:
    mol = Chem.MolFromInchi(x)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    fp_arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, fp_arr)
    fp_arr = np.array([int(x) for x in fp_arr], dtype=np.float32)
    fingerprints.append(fp_arr)
morgan_test = pd.DataFrame(fingerprints)
morgan_test["Toxicity"] = test["Toxicity"]

In [29]:
#macckey
fingerprint_mac = []
for x in data["Inchi"]:
    mol = Chem.MolFromInchi(x)
    fp = AllChem.GetMACCSKeysFingerprint(mol)
    fp_arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, fp_arr)
    fp_arr = np.array([int(x) for x in fp_arr], dtype=np.float32)
    fingerprint_mac.append(fp_arr)
mackey_data = pd.DataFrame(fingerprint_mac)
mackey_data["Toxicity"] = data["Toxicity"]

fingerprint_mac = []
for x in test["Inchi"]:
    mol = Chem.MolFromInchi(x)
    fp = AllChem.GetMACCSKeysFingerprint(mol)
    fp_arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, fp_arr)
    fp_arr = np.array([int(x) for x in fp_arr], dtype=np.float32)
    fingerprint_mac.append(fp_arr)
mackey_test = pd.DataFrame(fingerprint_mac)
mackey_test["Toxicity"] = test["Toxicity"]

[20:24:18] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match; Proton balance: Does not match)

[20:24:21] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Missing, Attachment points)

[20:24:21] WARNING: not removing hydrogen atom without neighbors
[20:24:21] WARNING: not removing hydrogen atom without neighbors


In [31]:
mordred_data.to_csv("data/mordred_data.csv")
mordred_test.to_csv("data/mordred_test.csv")
morgan_data.to_csv("data/morgan_data.csv")
morgan_test.to_csv("data/morgan_test.csv")
mackey_data.to_csv("data/mackey_data.csv")
mackey_test.to_csv("data/mackey_test.csv")